In [2]:
import os
import requests
import json
import pandas as pd
import difflib

In [61]:
base_url = 'https://api.data.gov/ed/collegescorecard/v1/'
data_set = 'schools.json?'
fields = ['school.name', 
          'school.address',
          'location.lat',
          'location.lon',
          'latest.school.operating',
          'latest.admissions.admission_rate.overall',
          'latest.student.size',
          'latest.student.demographics.men',
          'latest.student.demographics.women',
          'latest.cost.tuition.in_state',
          'latest.cost.tuition.out_of_state',
          'latest.cost.attendance.academic_year',
          'latest.cost.attendance.program_year',
          'latest.student.demographics.race_ethnicity.white',
          'latest.student.demographics.race_ethnicity.black',
          'latest.student.demographics.race_ethnicity.hispanic',
          'latest.student.demographics.race_ethnicity.asian',
          'latest.student.demographics.race_ethnicity.aian',
          'latest.student.demographics.race_ethnicity.nhpi',
          'latest.student.demographics.race_ethnicity.unknown',
          'latest.student.retention_rate',
          'latest.admissions.sat_scores.average.overall',
          'latest.admissions.act_scores.midpoint.cumulative',
          'latest.academics.program_percentage',
          'latest.academics.program.assoc',
          'latest.earnings.1_yr_after_completion.median',
          'latest.earnings.4_yrs_after_completion.median'
         ]
cred_level = [
                'latest.programs.cip_4_digit.credential.level', # causes error sometimes 24
                'latest.programs.cip_4_digit.title'  # causes error sometimes 25
            ]

options = '&per_page=100&page=65&sort=school.name' # repeat with increasing the page count until you get empty results
api_key = '&api_key='

request_url = base_url + data_set + '&fields=' + ','.join(fields) + options + api_key


In [62]:
colleges_dict = {}
error_list = []
for page in range(6600//10,): # 66 when doing 100 per page, 6543 total colleges 6544/2
    if page % 5 == 0:
        print(page)
    options = '&per_page=10&page=' + str(page) + '&sort=school.name'
    request_url = base_url + data_set + '&fields=' + ','.join(fields+cred_level) + options + api_key
    data = None
    try:
        data = requests.get(request_url).json()
    except:
        print('502 Error')
        request_url = base_url + data_set + '&fields=' + ','.join(fields) + options + api_key
        data = requests.get(request_url).json()
        error_list.append(data)

    
    for college in data['results']:
        try:
            colleges_dict[college['school.name']] = college
        except Exception as e:
            print(e)
            pass

0
5
10
15
20
25
30
35
40
45
50
55
60
65
70
75
80
85
90
95
100
105
110
115
120
125
130
135
140
145
150
155
160
165
170
175
180
185
190
195
200
205
210
215
220
225
230
235
240
245
250
255
260
265
270
275
280
285
290
295
300
305
310
315
320
325
330
335
340
345
350
355
360
365
370
375
380
385
390
395
400
405
410
415
420
425
430
435
440
445
450
455
460
465
470
475
480
485
490
495
500
505
510
515
520
525
530
535
540
545
550
555
560
565
570
575
580
585
590
595
600
605
610
615
620
625
630
635
640
645
650
655


In [63]:
data

{'metadata': {'page': 659, 'total': 6543, 'per_page': 10}, 'results': []}

In [59]:
colleges_dict['Amherst College']

{'latest.school.operating': 1,
 'latest.admissions.admission_rate.overall': 0.0874,
 'latest.student.size': 1971,
 'latest.student.demographics.men': 0.4855,
 'latest.student.demographics.women': 0.5145,
 'latest.cost.tuition.in_state': 61150,
 'latest.cost.tuition.out_of_state': 61150,
 'latest.cost.attendance.academic_year': 77411,
 'latest.cost.attendance.program_year': None,
 'latest.student.demographics.race_ethnicity.white': 0.3876,
 'latest.student.demographics.race_ethnicity.black': 0.103,
 'latest.student.demographics.race_ethnicity.hispanic': 0.1542,
 'latest.student.demographics.race_ethnicity.asian': 0.1517,
 'latest.student.demographics.race_ethnicity.aian': 0.0025,
 'latest.student.demographics.race_ethnicity.nhpi': 0.002,
 'latest.student.demographics.race_ethnicity.unknown': 0.0203,
 'latest.admissions.sat_scores.average.overall': 1469,
 'latest.admissions.act_scores.midpoint.cumulative': 33,
 'latest.earnings.1_yr_after_completion.median': 43914,
 'latest.earnings.4_yr

In [67]:
data_frame = pd.DataFrame.from_dict(colleges_dict,orient='index')
data_frame.loc['University of Arizona']

latest.school.operating                                                                     1
latest.admissions.admission_rate.overall                                               0.8727
latest.student.size                                                                   36560.0
latest.student.demographics.men                                                        0.4398
latest.student.demographics.women                                                      0.5602
                                                                  ...                        
latest.programs.cip_4_digit                 [{'title': 'Agriculture, General.', 'credentia...
school.name                                                             University of Arizona
school.address                                                         1401 E University Blvd
location.lat                                                                        32.232672
location.lon                                                

In [69]:
data_frame.to_csv('gov_data_all.csv', index = True, index_label = True)

In [42]:
error_list

[]

In [90]:
arr = []
for key in colleges_dict:
    print(colleges_dict[key]['latest.programs.cip_4_digit'])


colleges_dict

{}

##### 

In [5]:
df = pd.read_csv('all_data.csv', header = 0)

In [ ]:
copy = colleges_dict.copy()
new = {}
for key in copy.keys():
    d = {}
    keys = list(copy[key].keys())
    keys[0] = 'Sat Avg'
    keys[1] = 'Act Avg'
    keys[2] = 'Median 1 Year Salary'
    keys[3] = 'Median 4 Year Salary'
    for i in range(4, len(keys)-1):
        keys[i] = keys[i].split('.')[2:]
        if keys[i][0] == 'program_percentage':
            keys[i] = keys[i][1]
        else:
            keys[i] = ' '.join(keys[i][1:])
    keys = keys[:-1]
    vals = list(copy[key].values())
    for i in range(len(keys)):
        d[keys[i]] = vals[i]
    new[key.lower()] = d

    

In [6]:
gov_dict = {}

for key, val in new.items():
    if '-' in key:
        key = key.replace('-', ' ')
    if '&' in key:
        key = key.replace('& ', '')
    gov_dict[key] = val

In [7]:
name_list = df['Name'].tolist() # from CSV
not_found_list = []   # items from CSV not in the gov data
count = 0
for name in name_list:
    if name not in gov_dict:
        gov_name = None
        temp = name.split()
            
        if len(temp[-1]) == len(temp[:-1]):
            gov_name = ' '.join(temp[:-1])
        
        not_found_list.append((name,gov_name))  # creates tuple: (CSV Name, Gov Name)


for i in range(len(not_found_list)):
    pair = not_found_list[i]
    if 'alabama a m' in pair[0]:
        not_found_list[i] = (pair[0], 'alabama a m university')
    if 'florida a m' in pair[0]:
        not_found_list[i] = (pair[0], 'florida agricultural and mechanical university')
    if not_found_list[i][1] in gov_dict.keys():
        count += 1
len(not_found_list) - count

430

In [9]:
not_in_gov = []
for j in range(len(not_found_list)):
    pair = not_found_list[j]
    if not pair[1]:
        print(pair[0])
        temp = difflib.get_close_matches(pair[0], list(gov_dict.keys()))
        for i in range(len(temp)):
            print(f'{i}: {temp[i]}', end = ' | ')
        inp = input()
        if inp != '':
            print(temp[int(inp)])
            not_found_list[j] = (pair[0], temp[int(inp)])
        else:
            not_in_gov.append(not_found_list[j])
        print('\n'*3)

american university in bulgaria
0: american university | 1: american university of puerto rico | 2: national american university kings bay | 





american university of puerto rico bayamon
0: inter american university of puerto rico bayamon | 1: american university of puerto rico | 2: university of puerto rico bayamon | 





american university of puerto rico manati
0: american university of puerto rico | 1: edp university of puerto rico manati | 2: inter american university of puerto rico metro | 





anderson university indiana
0: anderson university | 1: chamberlain university indiana | 2: devry university virginia | 





anderson university south carolina
0: strayer university south carolina | 1: devry university north carolina | 2: strayer university north carolina | 

...

yeshiva d'monsey rabbinical college




yeshiva gedolah imrei yosef dpinka
0: yeshiva gedolah imrei yosef d'spinka | 1: yeshiva gedolah shaarei shmuel | 2: yeshiva gedolah kesser torah | 

 0


yeshiva gedolah imrei yosef d'spinka




yeshiva shaar hatorah
0: yeshivath shaar hatorah | 1: yeshiva chemdas hatorah | 2: yeshiva shaar ephraim | 

 0


yeshivath shaar hatorah




york college
0: taylor college | 1: ozarka college | 2: lyon college | 

In [13]:
file = open('edited_name_list.txt', 'w')
for pair in not_found_list:
    file.write(str(pair) + '\n')
file.close()

In [18]:
new_df = pd.DataFrame.from_dict(gov_dict, orient='index')
new_df

,Sat Avg,Act Avg,Median 1 Year Salary,Median 4 Year Salary,agriculture,resources,architecture,ethnic_cultural_gender,communication,communications_technology,...,assoc public_administration_social_service,assoc social_science,assoc construction,assoc mechanic_repair_technology,assoc precision_production,assoc transportation,assoc visual_performing,assoc health,assoc business_marketing,assoc history
a better u beauty barber academy,NaN,NaN,NaN,NaN,0.0,0.0000,0.0,0.0000,0.0,0.0000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
a t still university of health sciences,NaN,NaN,73982.0,101684.0,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
aaniiih nakoda college,NaN,NaN,24085.0,25808.0,0.0,0.0769,0.0,0.0385,0.0,0.0000,...,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0
abc adult school,NaN,NaN,NaN,NaN,0.0,0.0000,0.0,0.0000,0.0,0.0088,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
abc adult school cabrillo lane,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
z hair academy,NaN,NaN,20038.0,25611.0,0.0,0.0000,0.0,0.0000,0.0,0.0000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
zane state college,NaN,NaN,33119.0,33031.0,0.0,0.0235,0.0,0.0000,0.0,0.0000,...,1.0,0.0,1.0,0.0,1.0,0.0,0.0,2.0,2.0,0.0
zen shiatsu chicago,NaN,NaN,NaN,NaN,0.0,0.0000,0.0,0.0000,0.0,0.0000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
zion massage college,NaN,NaN,NaN,NaN,0.0,0.0000,0.0,0.0000,0.0,0.0000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [60]:
header = df.columns

20

In [76]:
df.insert(len(header), 'hello', 'a', True)

In [77]:
df = df.drop('hello', axis='columns')

In [78]:
df

,Name,Address,Geo Location,in state,out of state,First Year Students,Acceptance Rate (T/F/M),Total Applicants,Applicants Stats (F/M),Percent Accepted that Attend,Sex,Percent (F/M),Retention,International Students,Asian and Pacific Islander,Black,Hispanic,Native American,Other,White
0,aaniiih nakoda college,269 Blackfeet Avenue Harlem MT 59526,(48.48461 -108.75933),$2410,$2410,18.0,NaN,NaN,NaN,NaN,Co-ed,('n/a' 'n/a'),67%,0%,&lt; 1%,&lt; 1%,&lt; 1%,87%,&lt; 1%,13%
1,abilene christian university,1600 Campus Ct Abilene TX 79699,(32.46678 -99.70893),$50978,$50978,833.0,('70%' '71%' '66%'),8208.0,('65%' '35%'),14%,Co-ed,('62%' '38%'),77%,4%,1%,9%,18%,&lt; 1%,6%,66%
2,abraham baldwin agricultural college,2802 Moore Hwy Tifton GA 31793,(31.47547 -83.52372),$11527,$18433,865.0,('76%' '73%' '78%'),2190.0,('61%' '39%'),52%,Co-ed,('56%' '44%'),64%,1%,1%,21%,5%,&lt; 1%,1%,71%
3,academy college,1600 W. 82nd Street Suite 100 Bloomington MN 5...,(44.85602 -93.2996),$18644,$18644,16.0,NaN,NaN,NaN,NaN,Co-ed,('n/a' 'n/a'),100%,0%,4%,1%,2%,2%,33%,58%
4,academy of art university,79 New Montgomery San Francisco CA 94105,(37.78781 -122.40092),$44910,$44910,281.0,NaN,NaN,NaN,NaN,Co-ed,('n/a' 'n/a'),69%,24%,7%,12%,16%,1%,36%,27%
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2894,york technical college,452 S Anderson Rd Rock Hill SC 29730,(34.93594 -80.99659),$5410,$10354,615.0,NaN,NaN,NaN,NaN,Co-ed,('n/a' 'n/a'),57%,0%,2%,22%,9%,1%,8%,58%
2895,young harris college,1 College Street Young Harris GA 30582,(34.9333 -83.84719),$41664,$41664,304.0,('75%' '79%' '69%'),1850.0,('57%' '43%'),22%,Co-ed,('50%' '50%'),76%,6%,1%,7%,4%,&lt; 1%,17%,70%
2896,youngstown state university,One University Plaza Youngstown OH 44555,(41.10664 -80.64647),$19796,$20156,1627.0,('78%' '78%' '77%'),6718.0,('60%' '40%'),31%,Co-ed,('54%' '46%'),76%,3%,1%,11%,4%,&lt; 1%,8%,76%
2897,yuba college,2088 N Beale Rd Marysville CA 95901,(39.12804 -121.5397),$1128,$7488,311.0,NaN,NaN,NaN,NaN,Co-ed,('n/a' 'n/a'),38%,0%,17%,3%,39%,1%,10%,30%
